# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Step 2: Inspect the data
print(df.head())

# Step 3: Investigate the 'id' column
# Determine how many viewers also clicked
viewers = df[df['action'] == 'view']['id'].nunique()
clickers = df[df['action'] == 'click']['id'].nunique()
print("Number of viewers:", viewers)
print("Number of clickers:", clickers)

# Check for anomalies
anomalies = df[(df['action'] == 'click') & ~(df['id'].isin(df[df['action'] == 'view']['id']))]
print("Anomalies:", anomalies)

# Identify overlap between control and experiment groups
control_ids = df[df['group'] == 'control']['id'].unique()
experiment_ids = df[df['group'] == 'experiment']['id'].unique()
overlap = set(control_ids) & set(experiment_ids)
print("Overlap between control and experiment groups:", overlap)

# Step 4: Plan for experimental design
# Decide how to account for overlap (e.g., exclude overlapping IDs from analysis)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of viewers: 6328
Number of clickers: 1860
Anomalies: Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []
Overlap between control and experiment groups: set()


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [7]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Calculate click-through rates (CTR)
control_clicks = df[df['group'] == 'control']['action'].value_counts()['click']
experiment_clicks = df[df['group'] == 'experiment']['action'].value_counts()['click']
control_viewers = df[df['group'] == 'control']['action'].value_counts()['view']
experiment_viewers = df[df['group'] == 'experiment']['action'].value_counts()['view']
control_ctr = control_clicks / control_viewers
experiment_ctr = experiment_clicks / experiment_viewers

# Perform two-sample proportion z-test
z_stat, p_value = proportions_ztest([control_clicks, experiment_clicks], [control_viewers, experiment_viewers], alternative='larger')
print(p_value)
# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective than the control homepage.")
else:
    print("Fail to reject the null hypothesis. There is no evidence that the experimental homepage is more effective than the control homepage.")


0.9955849622117021
Fail to reject the null hypothesis. There is no evidence that the experimental homepage is more effective than the control homepage.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
#Your code here
# Assuming control_ctr is the click-through rate for the control group
# and experiment_viewers is the number of viewers in the experiment group

expected_clicks_experiment = control_ctr * experiment_viewers
print("Expected Clicks for Experiment Group:", expected_clicks_experiment)


Expected Clicks for Experiment Group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:


# Calculate actual number of clicks in the experiment group
actual_clicks_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate the standard deviation
std_dev_experiment = np.sqrt(experiment_viewers * control_ctr * (1 - control_ctr))

# Calculate the z-score
z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev_experiment
print("Number of Standard Deviations from Estimate:", z_score)


Number of Standard Deviations from Estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [6]:
#Your code here
from scipy.stats import norm

# Calculate the p-value using the cumulative distribution function (CDF)
p_value = 1 - norm.cdf(z_score)
print("p-value based on z-score:", p_value)


p-value based on z-score: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.